Notebook to generate config files for a given hyperparameter sweep.

In [1]:
import os
import yaml
import itertools
import numpy as np

In [2]:
# Specify a directory for experiment

#########################
sweep_experiment_name = "rattlesnakes_CRNN_model_selection"
sweep_experiment_dir_parent = "/home/jupyter/behavior_benchmarks_experiments"
model_type = 'CRNN'
dataset_dir = '/home/jupyter/behavior_data_local/data/formatted/desantis_rattlesnakes'
#########################

sweep_experiment_dir = os.path.join(sweep_experiment_dir_parent, sweep_experiment_name)
if not os.path.exists(sweep_experiment_dir):
    os.makedirs(sweep_experiment_dir)

In [3]:
# convenience function to generate combinations of parameters

def generate_choice_combinations(d):
    # d is a dict of lists
    # require that the keys of d are all of the same type (eg strings)
    # output a dict {i:{key:value} for i in range(N)}
    # which contains all possible combinations of values (encoded in the lists)
    # and N = product of len(d[key]), for all keys in d
    sorted_keys = sorted(d.keys())
    choices_lex = []
    num_choices = {}
    for key in sorted_keys:
        num_choices[key] = len(d[key])
        choices_lex.append(list(range(num_choices[key])))
    choices_cartesian = list(itertools.product(*choices_lex))
    configs_cartesian = {}
    for i, choices in enumerate(choices_cartesian):
        configs_cartesian[i] = {sorted_keys[j]: d[sorted_keys[j]][choices[j]] for j in range(len(sorted_keys))}
        
    return configs_cartesian

In [4]:
# Specify general parameters

sweep_config = {}

################
sweep_config['dataset_dir'] = [dataset_dir]
################

sweep_config['model'] = [model_type] # do not change
sweep_config['output_parent_dir'] = [sweep_experiment_dir]

summary = sweep_config.copy()

In [5]:
# Specify model-specific parameters
# For each model-specific parameter, possible values for the grid search are formatted as a list

####################

if model_type == 'vame':
    sweep_model_config = {'batch_size' : [512], 
                          'max_epochs' : [150],
                          'beta' : [0, 1], ## Scalar multiplied by KL loss
                          'zdims' : [20], ## Latent space dimensionality
                          'learning_rate' : [0.0005, 0.0001],
                          'time_window_sec' : [2, 5],
                          'prediction_decoder' : [1], ## Whether to predict future steps
                          'prediction_sec' : [2, 5], ## How much to predict after encoded window
                          'scheduler' : [1],
                          'scheduler_step_size' : [100],
                          'scheduler_gamma' : [0.2],
                          'kmeans_lambda' : [0.1],
                          'downsizing_factor' : [50]
                         }
    
    summary['vame_config'] = sweep_model_config
    
if model_type == 'hmm':
    sweep_model_config = {'time_bins' : [2500],
                          'prior_wait_sec' : [5., 10.],
                          'sticky_prior_strength' : [0., 1., 2.],
                          'N_iters' : [50],
                          'lags' : [0, 1, 2, 3]
                         }
        
    summary['vame_config'] = sweep_model_config
    
if model_type == 'CRNN':
    sweep_model_config = {'downsizing_factor' : [32],
                          'lr' : [0.01, 0.005, 0.001],
                          'weight_decay' : [0, 1e-3],
                          'n_epochs' : [100],
                          'hidden_size' : [64],
                          'temporal_window_samples' : [64], 
                          'batch_size' : [32],
                          'conv_depth' : [2],
                          'sparse_annotations' : [False],
                          'ker_size' : [7],
                          'dilation' : [1, 3],
                          'gru_depth' : [1],
                          'gru_hidden_size' : [64]
                         }
        
    summary['CRNN_config'] = sweep_model_config      
    
if model_type == 'umapper':
    sweep_model_config = {'morlet_w' : [1., 5., 10., 15],
                          'n_neighbors' : [16],
                          'min_dist' : [0],
                          'downsample' : [8]}
    
    summary['umapper_config'] = sweep_model_config
    
if model_type == 'kmeans':
    sweep_model_config = {'max_iter' : [500],
                          'wavelet_transform' : [True],
                          'whiten' : [False],
                          'morlet_w' : [1., 5., 10., 15.],
                          'n_wavelets' : [25], 
                          'downsample' : [1]
                         }
    
    summary['kmeans_config'] = sweep_model_config
    
if model_type == 'iic':
    sweep_model_config = {'downsizing_factor' : [512],
                          'lr' : [0.001],
                          'weight_decay' : [0],
                          'n_epochs' : [1000],
                          'hidden_size' : [64],
                          'temporal_window_samples' : [2048],
                          'batch_size' : [64],
                          'dropout' : [0],
                          'jitter_scale' : [0],
                          'blur_scale' : [0],
                          'context_window_samples' : [15, 51, 101],
                          'conv_depth' : [4],
                          'ker_size' : [7],
                          'dilation' : [1, 3, 5],
                          'n_heads' : [2]}
    summary['iic_config'] = sweep_model_config
    
if model_type == 'hubert_motion':
    sweep_model_config = {'downsizing_factor' : [5000],
                          'lr' : [3e-5, 1e-4, 3e-4],
                          'weight_decay' : [0],
                          'n_epochs' : [100],
                          'temporal_window_samples' : [10000], 
                          'batch_size' : [32],
                          'sparse_annotations' : [True],
                          'unfreeze_encoder' : [-1],
                          'normalize' : [False],
                          'model_path' : ['/home/jupyter/hubert_motion/models/beartest_extra_small_dynamic_no_wavelet/checkpoints/checkpoint_last.pt']
                         }
        
    summary['hubert_motion_config'] = sweep_model_config   
    
###########################

In [6]:
# Summarize what the experiment is about and save off
##########
summary['summary'] = "model selection"
##########
target_filename = sweep_experiment_name + "_summary" + '.yaml'
target_fp = os.path.join(sweep_experiment_dir, target_filename)                       
with open(target_fp, 'w') as file:
    yaml.dump(summary, file)
    
# Make cartesian combinations for sub-dictionaries
sweep_model_cartesian = generate_choice_combinations(sweep_model_config)

# Incorporate into main sweep dict
model_config_key_name = model_type + "_config"
sweep_config[model_config_key_name] = [sweep_model_cartesian[key] for key in sweep_model_cartesian]

# Make cartesian combinations:
sweep_config_cartesian = generate_choice_combinations(sweep_config)

# Number so as to get experiment names
for i in sweep_config_cartesian.keys():
    experiment_name = sweep_experiment_name + "_" + str(i)
    sweep_config_cartesian[i]['experiment_name'] = experiment_name
    
# save off configs:

config_fps = []
    
for i in sorted(sweep_config_cartesian.keys()):
    config = sweep_config_cartesian[i]
    target_filename = config['experiment_name'] + '.yaml'
    target_fp = os.path.join(sweep_experiment_dir, target_filename)
    config_fps.append(target_fp)                         
    with open(target_fp, 'w') as file:
        yaml.dump(config, file)

In [7]:
# Generate command line prompt to run
output = ""
for config_fp in config_fps:
    output += "python single_experiment.py --config " + config_fp + "; "

print(output)
#python train_model.py --config /home/jupyter/behavior_benchmarks/behavior_benchmarks/example_config/example_config_eskmeans.yaml

python full_experiment.py --config /home/jupyter/behavior_benchmarks_experiments/crows_vame_smooth_model_selection/crows_vame_smooth_model_selection_0.yaml; python full_experiment.py --config /home/jupyter/behavior_benchmarks_experiments/crows_vame_smooth_model_selection/crows_vame_smooth_model_selection_1.yaml; python full_experiment.py --config /home/jupyter/behavior_benchmarks_experiments/crows_vame_smooth_model_selection/crows_vame_smooth_model_selection_2.yaml; python full_experiment.py --config /home/jupyter/behavior_benchmarks_experiments/crows_vame_smooth_model_selection/crows_vame_smooth_model_selection_3.yaml; python full_experiment.py --config /home/jupyter/behavior_benchmarks_experiments/crows_vame_smooth_model_selection/crows_vame_smooth_model_selection_4.yaml; python full_experiment.py --config /home/jupyter/behavior_benchmarks_experiments/crows_vame_smooth_model_selection/crows_vame_smooth_model_selection_5.yaml; python full_experiment.py --config /home/jupyter/behavior_